### 実践演習 (85分)
#### 課題1: Chainerを用いたCNNの実装 (50分)
* MNISTデータセットを用いて、CNNの学習に挑戦
* 第6回までの知識と合わせて、正解率99%を目指そう
* 使えるツール：畳み込み層、プーリング層、Batch Normalization、SGD、Adam、Dropout

#### 課題2a：(パイロット用) Data Augmentation (20分)
* Horizontal flippingの実装 (行列のスライス)
* Scale augmentation (scipy.misc.imresize)
* Random crop/padding (np.pad)

# Import
お決まりのimport

https://docs.chainer.org/en/stable/tutorial/basic.html#core-concept

In [ ]:
import numpy as np
import chainer
from chainer import cuda, Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions

# 課題1 Chainerを用いたCNNの実装 

MNISTデータセットを使って、CNNの学習に挑戦

### [appendix] chainer公式documentを見よう！

どんなライブラリもそうですが、公式ドキュメント・公式チュートリアルが一番充実しているのです。

3-layers MLPによるMNIST学習
https://docs.chainer.org/en/stable/tutorial/basic.html#example-multi-layer-perceptron-on-mnist

いろいろな画像認識CNNのサンプルコード
https://docs.chainer.org/en/stable/tutorial/convnet.html

## MNISTデータセット

MNISTデータセットは、かつて画像認識のベンチマークとしてよく使われた、手書き文字データセットです。

28x28ピクセル・グレースケールの0~9の数字の手書き文字画像が、学習データ60000枚、テストデータ10000枚含まれています。

### MNISTデータセットの取得

データは http://yann.lecun.com/exdb/mnist/ から取得できます。特殊なファイルフォーマット（ページ下部に記載）なので、データ読み取りプログラムを自分でちょちょっと書きますorどこかから貰ってきます。

…というのが本来なのですが、chainerが便利メソッドを用意してくれています！

In [ ]:
train, test = datasets.get_mnist(ndim=3)

どんなデータなのか、少し見てみましょう

In [ ]:
import matplotlib.pyplot as plt
% matplotlib inline

sample_mnist = train[0]

plt.imshow(sample_mnist[0][0], cmap='gray')
print('Label:', sample_mnist[1])

### CNNで画像分類サンプル

In [ ]:
""" このモデルは講師がテキトーに作ったモデルなので、まだまだ良くできます！
コードの書き方の参考にしつつ、もっとよいモデルを作ろう！
(LeNet5とか試すだけでも大きく伸びます)
"""
class SampleCNN(Chain):
    def __init__(self):
        super(SampleCNN, self).__init__()
        with self.init_scope():
            self.conv1 = L.Convolution2D(in_channels=1, out_channels=20, ksize=5, stride=1, pad=0)
            self.conv2 = L.Convolution2D(in_channels=20, out_channels=50, ksize=5, stride=1, pad=0)
            self.l1 = L.Linear(None, 500)
            self.l2 = L.Linear(500, 10)
        
    def __call__(self, x):
        h = F.max_pooling_2d(self.conv1(x), 2, 2)
        h = F.max_pooling_2d(self.conv2(h), 2, 2)
        h = F.relu(self.l1(h))
        y = F.relu(self.l2(h))
        
        return y

In [ ]:
train_iter = iterators.SerialIterator(train, batch_size=100, shuffle=True)
test_iter = iterators.SerialIterator(test, batch_size=100, repeat=False, shuffle=False)

In [ ]:
model = L.Classifier(SampleCNN())

optimizer = optimizers.SGD()
optimizer.setup(model)

updater = training.StandardUpdater(train_iter, optimizer)
trainer = training.Trainer(updater, (20, 'epoch'), out='result')

trainer.extend(extensions.Evaluator(test_iter, model))
trainer.extend(extensions.LogReport())
trainer.extend(extensions.PrintReport(['epoch', 'main/accuracy', 'validation/main/accuracy']))
trainer.extend(extensions.ProgressBar())

trainer.run()

### Appendix: MLP
単純な3layers MLPで95%くらい出るよ！
CNNで99%を目指そう

In [ ]:
class MLP(Chain):
    def __init__(self, n_units, n_out):
        super(MLP, self).__init__()
        with self.init_scope():
            self.l1 = L.Linear(None, n_units)
            self.l2 = L.Linear(None, n_units)
            self.l3 = L.Linear(None, n_out)

    def __call__(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        y = self.l3(h2)
        return y

In [ ]:
model = L.Classifier(MLP(100, 10))

In [ ]:
""" LeNet5 """
class LeNet5(Chain):
    def __init__(self):
        super(LeNet5, self).__init__()
        with self.init_scope():
            self.conv1 = L.Convolution2D(
                in_channels=1, out_channels=6, ksize=5, stride=1)
            self.conv2 = L.Convolution2D(
                in_channels=6, out_channels=16, ksize=5, stride=1)
            self.conv3 = L.Convolution2D(
                in_channels=16, out_channels=120, ksize=4, stride=1)
            self.fc4 = L.Linear(None, 84)
            self.fc5 = L.Linear(84, 10)

    def __call__(self, x):
        h = F.sigmoid(self.conv1(x))
        h = F.max_pooling_2d(h, 2, 2)
        h = F.sigmoid(self.conv2(h))
        h = F.max_pooling_2d(h, 2, 2)
        h = F.sigmoid(self.conv3(h))
        h = F.sigmoid(self.fc4(h))
        return self.fc5(h)

# 課題2 Data Augmentation
Data Augmentationを実際にやってみましょう。

numpyの行列操作やscipy, scikit-imageなどのメソッドを駆使して画像をいじります

In [ ]:
from skimage.io import imread

In [ ]:
lena = imread('./lena_color.gif')

In [ ]:
plt.imshow(lena)

## Horizontal flippingの実装
目標：
<img src="images/lena_horizontal_flipped.png" width="200"/>

行列のスライス

ヒント(?)1
```python
x = np.array([1, 2, 3])
x[::-1]  # array([3, 2, 1])
```

ヒント(?)2
```python
x = np.array([
  [1, 2, 3],
  [4, 5, 6],
  [7, 8, 9],
])
x[:, 1]  # array([2, 5, 8])
```

In [ ]:
lena_hf = lena[:, ::-1, :]
plt.imshow(lena_hf)

In [ ]:
from skimage.io import imsave

imsave('./images/lena_horizontal_flipped.png', lena_hf)

## Scale augmentation
目標(例:1.2倍)：
<img src="./images/lena_zoom120.png" width="200"/>

scipy.misc.imresize

In [ ]:
from scipy.misc import imresize

resized_lena = imresize(lena, 1.2)
org_h, org_w = lena.shape[0:2]
h, w = resized_lena.shape[0:2]
sx = (w - org_w) // 2
sy = (h - org_h) // 2

resized_lena = resized_lena[sy:sy+org_h, sx:sx+org_w, :]
plt.imshow(resized_lena)

In [ ]:
imsave('./images/lena_zoom120.png', resized_lena)

## Random crop
目標（例）:

<img src="./images/lena_cropped_0.png" width="200"/>
<img src="./images/lena_cropped_1.png" width="200"/>
<img src="./images/lena_cropped_2.png" width="200"/>
<img src="./images/lena_cropped_3.png" width="200"/>

In [ ]:
resized_lena = imresize(lena, 1.5)
for i in range(4):
    sx = np.random.randint(0, w-org_w)
    sy = np.random.randint(0, h-org_h)
    cropped_lena = resized_lena[sy:sy+org_h, sx:sx+org_w, :]
    plt.imshow(cropped_lena)
    imsave('./images/lena_cropped_{}.png'.format(i), cropped_lena)